## Import Dependencies

In [1]:
# COCO related libraries
from samples.coco import coco

# MaskRCNN libraries
from mrcnn.config import Config



import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model_resnext_2_0_v3_nms as modellib

# Misc
import os
import sys
import json
import numpy as np
import time
from PIL import Image, ImageDraw

## Constants

## Additional setup

In [2]:
# Set the ROOT_DIR variable to the root directory of the Mask_RCNN git repo
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Select which GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

## Include evaluation scripts in training script so that the kernel does not have to be reloaded. Eases the process of rapidly training and evaluating models

## Import dependencies

In [3]:
# COCO related libraries
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils
from samples.coco import coco
from samples.coco.coco import evaluate_coco


# Misc
import os
import skimage.io
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tnrange, tqdm_notebook
%matplotlib inline 

In [4]:
# Number of classes in dataset. Must be of type integer
NUM_CLASSES = 5

# Relative path to .h5 weights file
#WEIGHTS_FILE = None

WEIGHTS_FILE = 'logs/mask_rcnn_5_classes_resnetxt_161_4x4_epc_250_step_per_ep_220_val_steps_100_alldataset_aug_v1_alpha_1_0219.h5'

# Relative path to ground truth annotations JSON file
#TEST_ANNOTATIONS_FILE ='datasets/aug_1/test/test.json'
TEST_ANNOTATIONS_FILE ='datasets/aug_1/test/test.json'


# Relative path to images associated with ground truth JSON file
TEST_DATASET_DIR = 'datasets/aug_1/test/'

# Relative path to the directory of images that you want to run inferencing on
TEST_IMAGE_DIR = 'datasets/aug_1/test/'

MODEL_NAME = "test"



## Declare evaluation configuration

In [5]:
class EvalConfig(coco.CocoConfig):
    """ Configuration for evaluation """
    
    # Give the configuration a recognizable name
    NAME = MODEL_NAME
    
    # How many GPUs
    GPU_COUNT = 1
    
    # How many images per gpu
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + NUM_CLASSES  # background + other classes
    
    #layers
    FPN_CLASSIF_FC_LAYERS_SIZE = 1024
    
    IMAGE_MIN_DIM = 1152
    IMAGE_MAX_DIM = 1280
    
    # Alpha for changing mask loss weights
    ALPHA =1.
    
    LOSS_WEIGHTS = {
        "rpn_class_loss": 1.,
        "rpn_bbox_loss": 1.,
        "mrcnn_class_loss": 1.,
        "mrcnn_bbox_loss": 1.,
        "mrcnn_mask_loss": ALPHA* 1.
    }    
    
    # Matterport originally used resnet101, but I downsized to fit it on my graphics card
    # ["resnet50", "resnet101", "resnet152", "resnet203", "resnetxt50", "resnetxt101", "resnetxt152",  "resnetxt203"]
    BACKBONE = 'resnetxt152'
    
    CARDINALITY = 32



    # To be honest, I haven't taken the time to figure out what these do
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    
    # Changed to 512 because that's how many the original MaskRCNN paper used
    TRAIN_ROIS_PER_IMAGE = 512#200
    MAX_GT_INSTANCES = 256#114
    POST_NMS_ROIS_INFERENCE = 2000#1000 
    POST_NMS_ROIS_TRAINING = 2000 
    
    DETECTION_MAX_INSTANCES = 400#114
    DETECTION_MIN_CONFIDENCE = 0.5

## Display configuration

In [6]:
EvalConfig().display()


Configurations:
ALPHA                          1.0
BACKBONE                       resnetxt152
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
CARDINALITY                    32
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1280
IMAGE_META_SIZE                18
IMAGE_MIN_DIM                  1152
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1280 1280    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcn

## Build class to load ground truth data

In [7]:
class CocoDataset(utils.Dataset):
    def load_coco_gt(self, annotations_file, dataset_dir):
        """Load a COCO styled ground truth dataset
        """
        
        # Create COCO object
        coco = COCO(annotations_file)

        # Load all classes
        class_ids = sorted(coco.getCatIds())

        # Load all images
        image_ids = list(coco.imgs.keys())

        # Add classes
        for i in class_ids:
            self.add_class("coco", i, coco.loadCats(i)[0]["name"])

        # Add images
        for i in image_ids:
            self.add_image(
                "coco", image_id = i,
                path = os.path.join(dataset_dir, coco.imgs[i]['file_name']),
                width = coco.imgs[i]["width"],
                height = coco.imgs[i]["height"],
                annotations = coco.loadAnns(coco.getAnnIds(
                    imgIds = [i], catIds = class_ids, iscrowd=None)))
        
        return coco
    
    def load_mask(self, image_id):
        """Load instance masks for the given image.
        Different datasets use different ways to store masks. This
        function converts the different mask format to one format
        in the form of a bitmap [height, width, instances].
        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a COCO image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "coco":
            return super(CocoDataset, self).load_mask(image_id)

        instance_masks = []
        class_ids = []
        annotations = self.image_info[image_id]["annotations"]
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.
        for annotation in annotations:
            class_id = self.map_source_class_id(
                "coco.{}".format(annotation['category_id']))
            if class_id:
                m = self.annToMask(annotation, image_info["height"],
                                   image_info["width"])
                # Some objects are so small that they're less than 1 pixel area
                # and end up rounded out. Skip those objects.
                if m.max() < 1:
                    continue
                # Is it a crowd? If so, use a negative class ID.
                if annotation['iscrowd']:
                    # Use negative class ID for crowds
                    class_id *= -1
                    # For crowd masks, annToMask() sometimes returns a mask
                    # smaller than the given dimensions. If so, resize it.
                    if m.shape[0] != image_info["height"] or m.shape[1] != image_info["width"]:
                        m = np.ones([image_info["height"], image_info["width"]], dtype=bool)
                instance_masks.append(m)
                class_ids.append(class_id)

        # Pack instance masks into an array
        if class_ids:
            mask = np.stack(instance_masks, axis=2).astype(np.bool)
            class_ids = np.array(class_ids, dtype=np.int32)
            return mask, class_ids
        else:
            # Call super class to return an empty mask
            return super(CocoDataset, self).load_mask(image_id)

    def image_reference(self, image_id):
        """Return a link to the image in the COCO Website."""
        info = self.image_info[image_id]
        if info["source"] == "coco":
            return "http://cocodataset.org/#explore?id={}".format(info["id"])
        else:
            super(CocoDataset, self).image_reference(image_id)

    # The following two functions are from pycocotools with a few changes.

    def annToRLE(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE to RLE.
        :return: binary mask (numpy 2D array)
        """
        segm = ann['segmentation']
        if isinstance(segm, list):
            # polygon -- a single object might consist of multiple parts
            # we merge all parts into one mask rle code
            rles = maskUtils.frPyObjects(segm, height, width)
            rle = maskUtils.merge(rles)
        elif isinstance(segm['counts'], list):
            # uncompressed RLE
            rle = maskUtils.frPyObjects(segm, height, width)
        else:
            # rle
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
        :return: binary mask (numpy 2D array)
        """
        rle = self.annToRLE(ann, height, width)
        m = maskUtils.decode(rle)
        return m

## Build MaskRCNN Model

In [8]:
# Create the model in inference mode
model = modellib.MaskRCNN(mode = "inference", config = EvalConfig(), model_dir = MODEL_DIR)

Cardinality, network, blocks:  32 resnetxt152 35
Metal device set to: Apple M1
prop Tensor("ROI/strided_slice_21:0", shape=(None, 4), dtype=float32)
soft nms Tensor("ROI/packed_2:0", shape=(1, None, 4), dtype=float32)
keep antes Tensor("mrcnn_detection/strided_slice_22:0", shape=(None,), dtype=int64)
keep Tensor("mrcnn_detection/strided_slice_24:0", shape=(None,), dtype=int64)
Instructions for updating:
Use fn_output_signature instead
N None
final class keep Tensor("mrcnn_detection/map/while/PadV2:0", shape=(400,), dtype=int64)
Tensor("mrcnn_detection/Pad:0", shape=(None, 6), dtype=float32)


## Load weights from last trained model

In [ ]:
if WEIGHTS_FILE is None:
    model.load_weights(model.find_last(), by_name = True)
else:
    model.load_weights(WEIGHTS_FILE, by_name = True)

## Load dataset

In [ ]:
dataset_val = CocoDataset()
coco = dataset_val.load_coco_gt(annotations_file = TEST_ANNOTATIONS_FILE, dataset_dir = TEST_DATASET_DIR)
dataset_val.prepare()
class_names = dataset_val.class_names

In [ ]:
print(class_names)

## Evaluate model with COCO test
### If your results come back as a bunch of zeros, check to make sure that the "width" and "height" tag in your COCO dataset are correct

In [ ]:
#evaluate_coco(model, dataset_val, coco, "segm")

## Calculating mAP as per example in train_shapes.ipynb

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, len(dataset_val.image_ids))

# Instanciate arrays to create our metrics
APs = []
ARs = []
precisions_arr = []
recalls_arr = []
overlaps_arr = []
class_ids_arr = []
scores_arr = []
F1_scores = []; 

for id in tnrange(len(image_ids), desc = "Processing images in dataset..."):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, EvalConfig(),
                               image_ids[id], use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, EvalConfig()), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    
    AR, positive_ids = utils.compute_recall(r["rois"], gt_bbox, iou=0.2)
    # Append AP to AP array
    APs.append(AP)
    ARs.append(AR)
    
    #F1_scores.append((2* (mean(precisions) * mean(recalls)))/(mean(precisions) + mean(recalls)))
    np.mean(APs)
    np.mean(ARs)
    
    # Append precisions
    for precision in precisions:
        precisions_arr.append(precision)
    
    # Append recalls
    for recall in recalls:
        recalls_arr.append(recall)
        
    
    
    # Append overlaps
    for overlap in overlaps:
        overlaps_arr.append(overlap)
    
    # Append class_ids
    for class_id in r["class_ids"]:
        class_ids_arr.append(class_id)
    
    # Append scores 
    for score in r["scores"]:
        scores_arr.append(score)
        
        
    
mAP =  np.mean(APs)
mAR = np.mean(ARs)
print("mAP: ", mAP)
print("mAR: ", mAR)


F1_score_2 = (2 * mAP * mAR)/(mAP + mAR)
print('second way calculate f1-score_2: ', F1_score_2)

In [ ]:
#Save the variables in a txt file 
file = open(MODEL_NAME+"/"+"variable.txt", "w")
file.write("mAP = " + str(mAP) + "\n" +"mAR = "+ str(mAR) + "\n"+"F1 = "+ str(F1_score_2) )
file.close()

## Plot precision recall curve

In [ ]:
visualize.plot_precision_recall(MODEL_NAME, AP, precisions, recalls)

# Confusion Matrix

In [ ]:
# an example of plotting confusion matrix.
# the first step consists of computing ground-truth and prediction vectors for all images.
# using these vectors, the plot_confusion_matrix_from_data function plots the CM and computes tps fps and fns
import pandas as pd
import numpy as np
import os 

#ground-truth and predictions lists
gt_tot = np.array([])
pred_tot = np.array([])
#mAP list
mAP_ = []


for id in tnrange(len(image_ids), desc = "Processing images in dataset..."):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, EvalConfig(),
                               image_ids[id], use_mini_mask=False)

    # Run the model
    results = model.detect([image], verbose=1)
    r = results[0]
    
    #compute gt_tot and pred_tot
    gt, pred = utils.gt_pred_lists(gt_class_id, gt_bbox, r['class_ids'], r['rois'])
    gt_tot = np.append(gt_tot, gt)
    pred_tot = np.append(pred_tot, pred)
    
    #precision_, recall_, AP_ 
    AP_, precision_, recall_, overlap_ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
    #check if the vectors len are equal
    print("the actual len of the gt vect is : ", len(gt_tot))
    print("the actual len of the pred vect is : ", len(pred_tot))
    
    mAP_.append(AP_)
    print("Average precision of this image : ",AP_)
    print("The actual mean average precision for the whole images (matterport methode) ", sum(mAP_)/len(mAP_))
   # print("Ground truth object : "+dataset_val.class_names[gt])
   # print("Predicted object : "+dataset_val.class_names[pred])


#print("ground truth list : ",gt_tot)
#print("predicted list : ",pred_tot)

tp,fp,fn, dm =utils.plot_confusion_matrix_from_data(MODEL_NAME, class_names, gt_tot,pred_tot,fz=18, figsize=(20,20), lw=0.5)

In [ ]:
image_ids

In [ ]:
# https://vitalflux.com/ml-metrics-sensitivity-vs-specificity-difference/

#Mathematically, sensitivity or true positive rate can be calculated as the following:

#Sensitivity = (True Positive)/(True Positive + False Negative)

Sensitivity =  sum(tp)/(sum(tp)+sum(fn))
print(Sensitivity)


# Mathematically, specificity can be calculated as the following:

# Specificity = (True Negative)/(True Negative + False Positive)
# 
#Save the variables in a txt file 
file = open(MODEL_NAME+"/"+"Sensitivity.txt", "w")
file.write("Sensitivity = " + str(Sensitivity)) 
file.close()

In [ ]:
print("tp for each class :",tp)
print("fp for each class :",fp)
print("fn for each class :",fn)

#eliminate the background class (class A) from tps fns and fns lists since it doesn't concern us anymore : 
del tp[0]
del fp[0]
del fn[0]


print("\n########################\n")
print("tp for each class :",tp)
print("fp for each class :",fp)
print("fn for each class :",fn)

In [ ]:
%cd ..
%cd home

In [ ]:
path_report = MODEL_NAME + '/' + str(NUM_CLASSES) +  'classes_images'

file_names = next(os.walk(TEST_IMAGE_DIR))[2]
file = open(path_report+"/"+"report_images.txt", "w")


for pic in file_names:
#image = skimage.io.imread(os.path.join(TEST_IMAGE_DIR, random.choice(file_names)))
    if pic.endswith('.jpg'):
        path_new = MODEL_NAME + '/' + str(NUM_CLASSES) +  'classes_images/' + str(pic)
        print(pic)
        image = skimage.io.imread(os.path.join(TEST_IMAGE_DIR, pic))
        #Save the variables in a txt file 
        file.write(str(pic)+"\n")
        file.write("--------------------------------------------"+"\n")


        # Run the model
        results = model.detect([image], verbose=1)
        r = results[0]
        flag = False

        for i in range(0, len(r['class_ids'])):
            if r['class_ids'][i] == 1:
                Xm = (r['rois'][i][1] + r['rois'][i][3])/2
                Ym = (r['rois'][i][0]+ r['rois'][i][2])/2
                coord = [Xm,Ym]
                print('A cancerigenous cell was found in X, Y: ', coord)
                file.write("'A cancerigenous cell was found in X, Y: '"+ str(coord)+"\n")
                flag = True



            if r['class_ids'][i] == 2:
                Xm = (r['rois'][i][1] + r['rois'][i][3])/2
                Ym = (r['rois'][i][0]+ r['rois'][i][2])/2
                coord = [Xm,Ym]
                print('A dangerous cell was found in X, Y: ', coord)     
                file.write("'A dangerous cell was found in X, Y: '"+ str(coord)+"\n")
                flag = True


        if flag == False:
            file.write("'No dangerous/cancerigenous cell was found'"+"\n")




        visualize.display_instances(path_new, image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
        file.write("\n")

file.close()
print("fim")

In [ ]:
import matplotlib.pyplot as plt
# Load a random image from the images folder
file_names = next(os.walk(TEST_IMAGE_DIR))[2]
rand = random.choice(file_names)
image = skimage.io.imread(os.path.join(TEST_IMAGE_DIR, rand))
path_new = MODEL_NAME + '/' + str(NUM_CLASSES) +  'classes_images/' + str(rand)

#image = skimage.io.imread("cell_2_4400_0_4900_500_jpg.rf.634fc6434c65b180a5c3a9bc993e1241.jpg")

# Run detection
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
visualize.display_instances(path_new, image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
